In [1]:
# to do : need to add padded batching suppport


import tensorflow as tf

from tensorflow.contrib.data import Dataset, Iterator
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc


In [2]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [3]:
audio_to_mfcc( '/home/saurabh/Documents/tf_orange/tf_orange/data/test2.wav').shape

(299, 13)

In [4]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
       print (type(parsed_line[1]))
       
       

       return parsed_line[0] , parsed_line[1]

In [55]:
import re

word_dictionary = {'hello':0 , 'world':1 , ' ':2}


def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list

In [56]:
def _read_py_function(audio, label):
    audio =audio_to_mfcc(audio)
   
    return audio ,label

In [57]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/data/data.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype])))


#dataset = dataset.padded_batch(2, padded_shapes=([299,13,None]))

dataset=dataset.batch(2)


<class 'tensorflow.python.framework.ops.Tensor'>


In [58]:
# create TensorFlow Iterator object
#iterator = Iterator.from_structure(tr_data.output_types,
 #                                  tr_data.output_shapes)
iterator = dataset.make_one_shot_iterator()

item = iterator.get_next()

In [59]:
#dataset = dataset.batch(3)

In [60]:
#training_init_op = iterator.make_initializer(tr_data)

In [63]:
with tf.Session() as sess:

    
    # get each element of the training dataset until the end is reached
    while True:
        try:
            elem = sess.run(item)
            print(elem[0].shape)
            for label in elem[1]:
                print (word_to_index(label.decode("utf-8")) )
                
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            break


(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
End of training dataset.
